# Evaluate MQA performance

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
pd.options.display.float_format = '{:.3f}'.format
plt.rcParams["figure.dpi"] = 150
sns.set(style='darkgrid')
from IPython.display import display
import warnings
warnings.simplefilter('ignore', UserWarning)
from pathlib import Path
from sklearn.metrics import mean_absolute_error
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams["font.size"] = 15
plt.rcParams['figure.figsize'] = (6, 4)

In [ ]:
data_dir = Path('../../../../../data/')
dataset_dir = data_dir / 'out' / 'dataset'
subset_name = 'target_subset_' + Path('.').resolve().parent.name
score_dir = dataset_dir / 'score' / 'subsets' / subset_name
assert score_dir.exists()
fig_dir = score_dir / 'fig' / 'MQA'
fig_dir.mkdir(parents=True, exist_ok=True)
target_list = data_dir / 'interim' / f'{subset_name}.csv'
domain_csv = data_dir / 'interim' / 'ecod_domain_num.csv'
assert target_list.exists()

In [ ]:
label_path = score_dir / 'label.csv'
label_df = pd.read_csv(label_path, index_col=0)
target_df = pd.read_csv(target_list, index_col=0)
neff_df = pd.read_csv(score_dir / 'neff.csv', index_col=0)
domain_df = pd.read_csv(domain_csv)
target_df = pd.merge(target_df, domain_df, on='id', how='left')
target_df = pd.merge(target_df, neff_df, left_on='id', right_on='Target').drop('id', axis=1)
label_df = pd.merge(label_df, target_df, on='Target', how='left')
label_df = label_df.drop(['pLDDT', 'pTMscore'], axis=1)
label_df.head()

In [ ]:
for i, csv in enumerate(sorted(score_dir.glob('*.csv.gz'))):
    if csv.name == 'all_score.csv.gz':
        continue
    method_df = pd.read_csv(csv, index_col=0)
    if csv.name in ['ProQ3D_resolved.csv.gz', 'DOPE_resolved.csv.gz', 'SBROD_resolved.csv.gz']:
        method_names = set(method_df.columns.tolist()) - {'Target', 'Model'}
        method_df = method_df.rename({method_name: f'{method_name}_resolved' for method_name in method_names}, axis=1)
    if i == 0:
        mqa_df = method_df
        continue
    mqa_df = pd.merge(mqa_df, method_df, on=['Model', 'Target'], how='outer')
mqa_df['DOPE_resolved'] = - mqa_df['DOPE_resolved']
mqa_df['SOAP_resolved'] = - mqa_df['SOAP_resolved']
mqa_df = mqa_df.rename({m: m.split('_')[0] for m in mqa_df.columns.tolist() if '_resolved' in m}, axis=1)
mqa_df

In [ ]:
df = pd.merge(label_df, mqa_df, on=['Target', 'Model'], how='left')
df_output_path = score_dir / 'all_score.csv.gz'
save_df = df.drop(['header', 'sequence'], axis=1)
save_df.to_csv(df_output_path, compression='gzip')
df

In [ ]:
import sys
sys.path.append('../../../../mqa')
from eval import eval, stat_test, eval_get_df

In [ ]:
mqa_df.columns

## Evaluate MQA peformance per target

### For GDT_TS

In [ ]:
# Against gdtts
mqa_methods = ['DOPE', 'SOAP', 'ProQ3D', 'SBROD', 'VoroCNN', 'Sato-3DCNN', 'P3CMQA', 'DeepAccNet', 'DeepAccNet-Bert']
methods = mqa_methods + ['pLDDT', 'pTM']
label = 'GDT_TS'
data = df
result_df = eval(data, methods, label_name=label)
result_df

In [ ]:
# Against gdtts
label = 'GDT_TS'
data = df.groupby('Target').filter(lambda x: x[label].max() - x[label].min() > 0.05)
result_each_target = eval_get_df(data, methods, label_name=label)
def get_whole_result_from_each_target(result_each_target, columns):
    result_df = result_each_target.groupby('Method').mean().reset_index()
    order_dict = dict(zip(columns, range(len(columns))))
    result_df['order'] = result_df['Method'].apply(lambda x: order_dict[x])
    result_df = result_df.sort_values('order')
    result_df = result_df.set_index('Method')
    result_df = result_df.reset_index().drop('order', axis=1)
    return result_df
result_df = get_whole_result_from_each_target(result_each_target, methods + ['Random selection'])
result_each_target = pd.merge(result_each_target, target_df, on='Target', how='left')
# result_each_target.to_csv(score_dir / 'mqa_result_each_target_gdtts_5.csv')
result_df

In [ ]:
result_each_target.query('Method == "Random selection"')[['Target', 'GDT_TS Loss']].sort_values('GDT_TS Loss')

### For models with pTMscore

In [ ]:
# Against gdtts (For models by ptm models)
label = 'GDT_TS'
data = df[~df['pTM'].isna()].groupby('Target').filter(lambda x: x[label].max() - x[label].min() > 0.05)
# excepted_targets = ['6Z4U_A', '6NEK_A']
# data = data.query('Target not in @excepted_targets')
result_each_target_ptm = eval_get_df(data, methods, label_name=label)
result_df_ptm = get_whole_result_from_each_target(result_each_target_ptm, methods + ['Random selection'])
result_each_target_ptm = pd.merge(result_each_target_ptm, target_df, on='Target', how='left')
result_df_ptm

In [ ]:
# Against gdtts (For models by ptm models)
label = 'GDT_TS'
metric = 'Loss'
sns.boxplot(data=result_each_target_ptm, x=f'{label} {metric}', y=f'Method', order=methods + ['Random selection'], color='white')
sns.swarmplot(data=result_each_target_ptm, x=f'{label} {metric}', y=f'Method', order=methods + ['Random selection'], size=2, palette='deep')
xticks = np.arange(-1, 6) * 10
xticks_start = -6
xticks[0] = xticks_start
xticks_str = xticks.astype(str)
xticks_str[0] = 'Mean'
plt.xlim(-11, 60)
plt.xticks(xticks, xticks_str)
plt.axvline(x=xticks_start, color=(234/255,234/255,242/255))
for i, method in enumerate(methods + ['Random selection']):
    value = result_df_ptm.query('Method == @method')[f'{label} {metric}'].values[0]
    plt.text(xticks_start, i, f'{value:.3f}', size=10, horizontalalignment='center', verticalalignment='center')
plt.tight_layout()
# plt.savefig(fig_dir / f'boxplot_{label}_{metric}_with_ptm.png')

In [ ]:
metrics = ['Pearson', 'Spearman']
data = result_each_target_ptm
label = 'GDT_TS'
y = 'Method'
for metric in metrics:
    x = f'{label} {metric}'
    sns.boxplot(data=data, x=x, y=y, order=methods, color='white')
    sns.swarmplot(data=data, x=x, y=y, order=methods, size=3)
    xticks_max = 1.2
    plt.xlim(-1, 1.35)
    xticks = np.append(np.arange(-1.0, 1.2, 0.5), [xticks_max])
    xticks_str = list(map(lambda x: f'{x:.1f}', xticks))
    xticks_str[-1] = 'Mean'
    plt.xticks(xticks, xticks_str)
    plt.axvline(x=xticks_max, color=(234/255,234/255,242/255))
    mean_series = data.groupby(y).mean()[x]
    for i, method in enumerate(methods):
        value = mean_series[method]
        plt.text(xticks_max, i, f'{value:.3f}', size=10, horizontalalignment='center', verticalalignment='center')
    plt.tight_layout()
    # plt.savefig(fig_dir / f'boxplot_{label}_{metric}_ptm.png')
    plt.show()

In [ ]:
label = 'GDT_TS'
metric = 'Loss'
plt.figure(figsize=(8, 5))
order = [True, False]
data = result_each_target_ptm
sns.boxplot(data=data, x=f'{label} {metric}', y=f'Method', order=methods + ['Random selection'], hue='is_similar_AF2', color='white', hue_order=order)
sns.swarmplot(data=data, x=f'{label} {metric}', y=f'Method', order=methods + ['Random selection'], size=1.5, hue='is_similar_AF2', dodge=True, hue_order=order)
plt.tight_layout()
# plt.savefig(fig_dir / f'boxplot_{label}_{metric}_similar_af2.png')
plt.show()

In [ ]:
# Against gdtts (For models by ptm models)
labels = ['GDT_TS', 'Mean_LDDT', 'TMscore', 'GDT_HA']
metrics = ['Pearson', 'Spearman', 'Loss', 'MAE']
result_each_target_ptms = []
for label in labels:
    data = df[~df['pTM'].isna()].groupby('Target').filter(lambda x: x[label].max() - x[label].min() > 0.05)
    result_each_target_ptm = eval_get_df(data, methods, label_name=label)
    result_df_ptm = get_whole_result_from_each_target(result_each_target_ptm, methods + ['Random selection'])
    display(result_df_ptm)
    result_each_target_ptm = result_each_target_ptm.rename(columns={f'{label} {metric}': f'{metric}' for metric in metrics})
    result_each_target_ptm['Label'] = label
    result_each_target_ptms.append(result_each_target_ptm)
result_each_target_ptm_each_label = pd.concat(result_each_target_ptms)
result_each_target_ptm_each_label = pd.merge(result_each_target_ptm_each_label, target_df, on='Target', how='left')
result_each_target_ptm_each_label.to_csv(score_dir / 'mqa_result_each_target_all_label_5_with_ptm.csv')

### MQA performance difference between is_similar_AF2

In [ ]:
# Against gdtts
label = 'GDT_TS'
for name, group in df[~df['pTM'].isna()].groupby('is_similar_AF2'):
    print(name)
    data = group
    result_df = eval(data, methods, label_name=label)
    display(result_df)

In [ ]:
# Against gdtts (For targets whose value difference between max and min is larger than 0.05)
label = 'GDT_TS'
for name, group in df[~df['pTM'].isna()].groupby('is_similar_AF2'):
    print(name)
    data = group.groupby('Target').filter(lambda x: x[label].max() - x[label].min() > 0.05)
    result_df = eval(data, methods, label_name=label)
    display(result_df)

### For Mean_LDDT

In [ ]:
# Against Mean LDDT
label = 'Mean_LDDT'
data = df[~df['pTM'].isna()].groupby('Target').filter(lambda x: x[label].max() - x[label].min() > 0.05)
result_each_target = eval_get_df(data, methods, label_name=label)
result_df = get_whole_result_from_each_target(result_each_target, methods + ['Random selection'])
result_each_target = pd.merge(result_each_target, target_df, on='Target', how='left')
result_df

In [ ]:
label = 'Mean_LDDT'
metric = 'Loss'
sns.boxplot(data=result_each_target, x=f'{label} {metric}', y=f'Method', order=methods + ['Random selection'], color='white')
sns.swarmplot(data=result_each_target, x=f'{label} {metric}', y=f'Method', order=methods + ['Random selection'], size=2, palette='deep')
xticks = np.arange(-1, 6) * 10
xticks_start = -6
xticks[0] = xticks_start
xticks_str = xticks.astype(str)
xticks_str[0] = 'Mean'
plt.xlim(-11, 60)
plt.xticks(xticks, xticks_str)
plt.axvline(x=xticks_start, color=(234/255,234/255,242/255))
# put parentheses around the pTMscore
yticks, yticks_texts = plt.yticks()
for i, ytick_text in enumerate(yticks_texts):
    if ytick_text.get_text() == 'pTMscore':
        yticks_texts[i] = '(pTMscore)'
plt.yticks(yticks, yticks_texts)
for i, method in enumerate(methods + ['Random selection']):
    value = result_df.query('Method == @method')[f'{label} {metric}'].values[0]
    plt.text(xticks_start, i, f'{value:.3f}', size=10, horizontalalignment='center', verticalalignment='center')
plt.tight_layout()
# plt.savefig(fig_dir / f'boxplot_{label}_{metric}.png')

In [ ]:
metrics = ['Pearson', 'Spearman']
data = result_each_target
label = 'Mean_LDDT'
y = 'Method'
for metric in metrics:
    x = f'{label} {metric}'
    sns.boxplot(data=data, x=x, y=y, order=methods, color='white')
    sns.swarmplot(data=data, x=x, y=y, order=methods, size=3)
    xticks_max = 1.2
    plt.xlim(-1, 1.35)
    xticks = np.append(np.arange(-1.0, 1.2, 0.5), [xticks_max])
    xticks_str = list(map(lambda x: f'{x:.1f}', xticks))
    xticks_str[-1] = 'Mean'
    plt.xticks(xticks, xticks_str)
    plt.axvline(x=xticks_max, color=(234/255,234/255,242/255))
    mean_series = data.groupby(y).mean()[x]
    for i, method in enumerate(methods):
        value = mean_series[method]
        plt.text(xticks_max, i, f'{value:.3f}', size=10, horizontalalignment='center', verticalalignment='center')
    plt.tight_layout()
    # plt.savefig(fig_dir / f'boxplot_{label}_{metric}.png')
    plt.show()

## Scatter plot between labels and MQA methods

In [ ]:
# GDT_TS
label = 'GDT_TS'
ms = ['ProQ3D', 'VoroCNN', 'Sato-3DCNN', 'P3CMQA', 'DeepAccNet', 'DeepAccNet-Bert', 'pLDDT', 'pTM']
# Fig size
ncols = 4
nrows = len(ms) // ncols
xwidth = 22
ywidth = xwidth * nrows / ncols
figsize = (xwidth, ywidth)
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex=False, sharey=False, figsize=figsize)
data = df
for i, method in enumerate(ms):
    ax = axes[i // ncols, i % ncols]
    sns.scatterplot(data=data, x=method, y=label, s=1, ax=axes[i // ncols, i % ncols])
    d = data[~data[method].isna()]
    x = d[method]
    y = d[label]
    corr = d.corr()[method][label]
    mae = mean_absolute_error(x, y)
    ax.set_xlabel(f'{method} (Cc: {corr:.3f}, MAE: {mae:.3f})')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    x_latent = np.linspace(x.min(), x.max(), 100)
    a, b = np.polyfit(x, y, 1)
    y_latent = a * x_latent + b
    ax.plot(x_latent, y_latent, 'r-')
    ax.legend(['Linear fitting'])
for i in range(len(ms), ncols * nrows):
    ax = axes[i // ncols, i % ncols]
    ax.axis('off')
plt.tight_layout()
# plt.savefig(fig_dir / f'scatterplot_methods_{label}.png')
plt.show()

In [ ]:
# GDT_TS with pTMscore
label = 'GDT_TS'
ms = ['ProQ3D', 'VoroCNN', 'Sato-3DCNN', 'P3CMQA', 'DeepAccNet', 'DeepAccNet-Bert', 'pLDDT', 'pTM']
# Fig size
ncols = 4
nrows = len(ms) // ncols
xwidth = 22
ywidth = xwidth * nrows / ncols
figsize = (xwidth, ywidth)
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex=False, sharey=False, figsize=figsize)
data = df[~df['pTM'].isna()]
for i, method in enumerate(ms):
    ax = axes[i // ncols, i % ncols]
    sns.scatterplot(data=data, x=method, y=label, s=1, ax=axes[i // ncols, i % ncols])
    d = data[~data[method].isna()]
    x = d[method]
    y = d[label]
    corr = d.corr()[method][label]
    mae = mean_absolute_error(x, y)
    ax.set_xlabel(f'{method} (Cc: {corr:.3f}, MAE: {mae:.3f})')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    x_latent = np.linspace(x.min(), x.max(), 100)
    a, b = np.polyfit(x, y, 1)
    y_latent = a * x_latent + b
    ax.plot(x_latent, y_latent, 'r-')
    ax.legend(['Linear fitting'])
for i in range(len(ms), ncols * nrows):
    ax = axes[i // ncols, i % ncols]
    ax.axis('off')
plt.tight_layout()
# plt.savefig(fig_dir / f'scatterplot_methods_{label}_with_ptm.png')
plt.show()

In [ ]:
# TMscore
label = 'TMscore'
# Fig size
ncols = 4
nrows = len(ms) // ncols
xwidth = 22
ywidth = xwidth * nrows / ncols
figsize = (xwidth, ywidth)
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex=False, sharey=False, figsize=figsize)
data = df[~df['pTM'].isna()]
for i, method in enumerate(ms):
    ax = axes[i // ncols, i % ncols]
    sns.scatterplot(data=data, x=method, y=label, s=1, ax=axes[i // ncols, i % ncols])
    d = data[~data[method].isna()]
    x = d[method]
    y = d[label]
    corr = d.corr()[method][label]
    mae = mean_absolute_error(y, x)
    ax.set_xlabel(f'{method} (Cc: {corr:.3f}, MAE: {mae:.3f})')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    x_latent = np.linspace(x.min(), x.max(), 100)
    a, b = np.polyfit(x, y, 1)
    y_latent = a * x_latent + b
    ax.plot(x_latent, y_latent, 'r-')
    ax.legend(['Linear fitting'])
for i in range(len(ms), ncols * nrows):
    ax = axes[i // ncols, i % ncols]
    ax.axis('off')
plt.tight_layout()
# plt.savefig(fig_dir / f'scatterplot_methods_{label}.png')
plt.show()

In [ ]:
# Mean LDDT
label = 'Mean_LDDT'
# Fig size
ncols = 4
nrows = len(ms) // ncols
xwidth = 22
ywidth = xwidth * nrows / ncols
figsize = (xwidth, ywidth)
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex=False, sharey=False, figsize=figsize)
data = df[~df[label].isna()]
data = data[~data['pTM'].isna()]
for i, method in enumerate(ms):
    ax = axes[i // ncols, i % ncols]
    sns.scatterplot(data=data, x=method, y=label, s=1, ax=axes[i // ncols, i % ncols])
    d = data[~data[method].isna()]
    x = d[method]
    y = d[label]
    corr = d.corr()[method][label]
    mae = mean_absolute_error(x, y)
    ax.set_xlabel(f'{method} (Cc: {corr:.3f}, MAE: {mae:.3f})')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    x_latent = np.linspace(x.min(), x.max(), 100)
    a, b = np.polyfit(x, y, 1)
    y_latent = a * x_latent + b
    ax.plot(x_latent, y_latent, 'r-')
    ax.legend(['Linear fitting'])
for i in range(len(ms), ncols * nrows):
    ax = axes[i // ncols, i % ncols]
    ax.axis('off')
plt.tight_layout()
# plt.savefig(fig_dir / f'scatterplot_methods_{label}.png')
plt.show()